### Model równoległy Conv 1D / RNN dla Zero Crossing Rate + RMS Energy

#### Architektura zawartego w tym notebooku modelu łączy dwie popularne ścieżki przetwarzania sygnałów audio: jednowymiarowe warstwy konwolucyjne (Conv 1D) i sieci rekurencyjne (RNN). Obie sieci działają równolegle. Wejściem dla modelu są cechy ZCR i RMS Energy obliczone dla każdej ramki czasowej sygnału audio. 

Etapy tworzenia modelu:

1. Przetworzenie danych wejściowych:

- Ekstrakcja cech Zero Crossing Rate (ZCR) i RMS Energy z plików audio:  
<ins> Zero Crossing Rate (ZCR) </ins> mierzy, jak często amplituda sygnału przechodzi z wartości dodatniej na ujemną lub z ujemnej na dodatnią. Określa liczbę przejść przez zero na jednostkę czasu, zazwyczaj na ramkę analizy. Na przykład wyższa wartość ZCR wskazuje na dźwięki wysokoczęstotliwościowe (np. syczące spółgłoski 's', 'f'), natomiast niższa wartość jest charakterystyczna dla dźwięków niskoczęstotliwościowych (np. samogłoski).  
<ins> RMS Energy </ins> to miara mocy lub głośności sygnału audio. Oblicza się ją jako pierwiastek kwadratowy ze średniej kwadratów amplitud sygnału w analizowanej ramce czasowej. Wyższa wartość wskazuje na głośniejszy dźwięk, niższa wartość na cichszy dźwięk lub ciszę.  
- Normalizacja i ujednolicenie długości sekwencji cech.


2. Budowa równoległej architektury:
- Ścieżka Conv1D wyodrębnia lokalne wzorce i cechy z sekwencji ZCR i RMS Energy, wykrywając krótkoterminowe zależności i charakterystyczne wzorce energii/przejść przez zero.
- Ścieżka RNN dobrze radzi sobie z modelowaniem długoterminowych zależności czasowych i kontekstu w sekwencjach ZCR i RMS Energy, co jest kluczowe dla rozpoznawania wzorców akustycznych.
- Obie ścieżki łączą się po ekstrakcji cech.

3. Implementacja funkcji pomocniczych:

- Wizualizacja wyników treningu, macierzy pomyłek.
- Obliczenie metryk klasyfikacji.
- Zapisanie modelu i mapowanie etykiet.


Przedstawiona architektura pozwala na równoległe przetwarzanie różnych rodzajów cech akustycznych, co może wpłynąć na otrzymanie lepszych wyników dla rozpoznawania emocji, ponieważ:

- Conv1D dobrze radzi sobie z lokalnymi wzorcami w sygnale (np. charakterystyczne częstotliwości przejść dla różnych emocji).
- BiLSTM potrafi uchwycić długoterminowe zależności i dynamiczne zmiany w energii sygnału.

### Import bibliotek

In [6]:
import sys
import os

# Dodaj katalog główny projektu do sys.path
current_dir = (
    os.path.dirname(os.path.abspath(__file__))
    if "__file__" in globals()
    else os.getcwd()
)
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Katalog główny projektu: {project_root}")
print(f"Czy katalog src istnieje: {os.path.exists(os.path.join(project_root, 'src'))}")

model_output_dir = os.path.join(project_root, "src", "CONV1D_RNN", "model_outputs")
os.makedirs(model_output_dir, exist_ok=True)
print(f"Wyniki modelu będą zapisywane w: {model_output_dir}")

import librosa
import librosa.display
import numpy as np
from datasets import load_from_disk, load_dataset
import matplotlib.pyplot as plt
import random
import json

# Klasyfikacja ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Deep Learning (PyTorch)
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Wizualizacja
import seaborn as sns

Katalog główny projektu: c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition
Czy katalog src istnieje: True
Wyniki modelu będą zapisywane w: c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\Conv1D_RNN\model_outputs


### Wczytanie zbioru nEMO

In [ ]:
# Weryfikacja istnienia folderu z danymi oraz załadowanie zbioru danych
try:
    from src.config import DATASET_PATH
    from src.create_data import download_and_save_dataset

    dataset_path = DATASET_PATH
    if os.path.exists(dataset_path):
        try:
            print("Rozpoczęcie ładowania datasetu z dysku...")
            dataset = load_from_disk(dataset_path)
        except Exception as e:
            print(f"Wystąpił błąd podczas ładowania datasetu: {e}")
            print("Inicjowanie ponownego pobierania datasetu...")
            dataset = download_and_save_dataset()
    else:
        print("Folder 'data' nie został znaleziony. Inicjowanie pobierania datasetu...")
        dataset = download_and_save_dataset()
except ImportError:
    print(
        "Moduły src.config i src.create_data nie są dostępne. Ładowanie przykładowego datasetu z HuggingFace."
    )
    dataset = load_dataset("amu-cai/nEMO", split="train")

# Sprawdź strukturę datasetu i wybierz właściwy split
if hasattr(dataset, "keys"):
    print(f"Dostępne splits: {list(dataset.keys())}")
    # Użyj split 'train' jeśli dostępny
    if "train" in dataset:
        dataset = dataset["train"]
        print("Używam split 'train'")
    else:
        # Jeśli nie ma 'train', użyj pierwszego dostępnego
        first_key = list(dataset.keys())[0]
        dataset = dataset[first_key]
        print(f"Używam split '{first_key}'")

print(f"Rozmiar datasetu: {len(dataset)}")
print(f"Przykładowy sample: {dataset[0] if len(dataset) > 0 else 'Brak danych'}")

### Zastosowanie augmentacji plików audio

In [ ]:
"""Dodaje biały szum do audio."""


def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio


"""Rozciąga czas (przyspiesza lub spowalnia) audio."""


def time_stretch(audio, rate=None):
    if rate is None:
        # Losowy współczynnik między 0.85 a 1.15
        rate = np.random.uniform(0.85, 1.15)
    return librosa.effects.time_stretch(y=audio, rate=rate)


"""Zmienia wysokość tonu audio."""


def pitch_shift(audio, sr, n_steps=None):
    if n_steps is None:
        # Losowa zmiana tonu w zakresie od -2 do 2 półtonów
        n_steps = np.random.uniform(-2, 2)
    return librosa.effects.pitch_shift(y=audio, sr=sr, n_steps=n_steps)


"""Zmienia głośność audio."""


def change_volume(audio, volume_factor=None):
    if volume_factor is None:
        # Losowy współczynnik głośności między 0.8 a 1.2
        volume_factor = np.random.uniform(0.8, 1.2)
    return audio * volume_factor


"""Aplikuje losowe augmentacje z określonym prawdopodobieństwem."""


def apply_augmentation(audio, sr, augment_prob=0.5):
    augmented = np.copy(audio)
    # Dodaj szum
    if random.random() < augment_prob:
        augmented = add_noise(augmented, noise_factor=random.uniform(0.001, 0.01))
    # Rozciągnij czas
    if random.random() < augment_prob:
        augmented = time_stretch(augmented)
    # Zmień wysokość tonu
    if random.random() < augment_prob:
        augmented = pitch_shift(augmented, sr)
    # Zmień głośność
    if random.random() < augment_prob:
        augmented = change_volume(augmented)
    return augmented

Dodane techniki augmentacji:

- Dodawanie szumu - Symuluje różne warunki nagrywania, zwiększając odporność modelu na zakłócenia.
- Zmiana tempa (time stretching) - Zmienia tempo audio bez zmiany wysokości tonu.
- Zmiana wysokości tonu (pitch shifting) - Zmienia ton audio bez zmiany tempa.
- Zmiana głośności - Symuluje różne odległości od mikrofonu.

Korzyści z augmentacji danych:

- Zwiększona ilość danych treningowych - Każda próbka audio jest augmentowana 2-krotnie, co trzykrotnie zwiększa rozmiar zbioru treningowego.
- Lepsza generalizacja - Model będzie odporniejszy na różne warunki nagrywania i różnice w mówieniu.
- Redukcja przeuczenia - Większa różnorodność danych pomaga modelowi skupić się na istotnych cechach, a nie na szumach specyficznych dla danego nagrania.
- Zrównoważenie klas - Augmentacja może częściowo pomóc w problemie niezrównoważenia klas.

Dodatkowo, kod zawiera funkcję wizualizacji efektów augmentacji, która pomaga zobaczyć, jak różne techniki wpływają na sygnał audio i ekstrahowane cechy (ZCR i RMS Energy).
Augmentacja danych jest szczególnie ważna w przypadku rozpoznawania emocji z głosu, ponieważ ten sam tekst wypowiedziany z tą samą emocją może brzmieć zupełnie inaczej u różnych osób lub w różnych warunkach nagrywania. Dzięki augmentacji uczymy model rozpoznawać wzorce emocji niezależnie od tych zmiennych czynników.

### Wizualizacja efektów augmentacji

In [ ]:
def extract_audio_features(audio_array, sr=24000, frame_length=1024, hop_length=256):
    # Normalizacja amplitudy
    audio_array = audio_array / (np.max(np.abs(audio_array)) + 1e-10)

    # Ekstrakcja Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(
        y=audio_array, frame_length=frame_length, hop_length=hop_length
    )

    # Ekstrakcja RMS Energy
    rms = librosa.feature.rms(
        y=audio_array, frame_length=frame_length, hop_length=hop_length
    )

    return zcr[0], rms[0]  # Zwracamy jako 1D array


# Przygotowanie danych
audio_files = [sample["audio"]["array"] for sample in dataset]
sample_rates = [sample["audio"]["sampling_rate"] for sample in dataset]
labels = [sample["emotion"] for sample in dataset]

# Konwertowanie etykiet emocji na liczby
emotion_labels = {emotion: idx for idx, emotion in enumerate(sorted(set(labels)))}
numeric_labels = [emotion_labels[label] for label in labels]

print("Mapowanie etykiet emocji:")
for emotion, idx in emotion_labels.items():
    print(f" {emotion}: {idx}")

# Ekstrakcja cech audio i przygotowanie danych
max_sequence_length = 200  # Maksymalna długość sekwencji cech
zcr_sequences = []
rms_sequences = []
aug_labels = []  # Etykiety z uwzględnieniem augmentowanych próbek

# Dla każdej próbki audio
for audio, sr, label in zip(audio_files, sample_rates, numeric_labels):
    # Przycinanie lub paddowanie audio do stałej długości (5 sekund)
    max_len = 5 * sr
    if len(audio) > max_len:
        start = np.random.randint(0, len(audio) - max_len)
        audio = audio[start : start + max_len]
    else:
        padding = np.zeros(max_len - len(audio))
        audio = np.concatenate([audio, padding])

    # Dodanie oryginalnej próbki
    zcr, rms = extract_audio_features(audio, sr=sr)

    # Dostosowanie długości sekwencji
    if len(zcr) > max_sequence_length:
        zcr = zcr[:max_sequence_length]
        rms = rms[:max_sequence_length]
    else:
        pad_len = max_sequence_length - len(zcr)
        zcr = np.pad(zcr, (0, pad_len), "constant")
        rms = np.pad(rms, (0, pad_len), "constant")

    zcr_sequences.append(zcr)
    rms_sequences.append(rms)
    aug_labels.append(label)

    # Dodanie augmentowanych próbek dla każdej emocji
    # Augmentujemy każdą próbkę 2 razy
    for _ in range(2):
        augmented_audio = apply_augmentation(audio, sr)
        aug_zcr, aug_rms = extract_audio_features(augmented_audio, sr=sr)

        if len(aug_zcr) > max_sequence_length:
            aug_zcr = aug_zcr[:max_sequence_length]
            aug_rms = aug_rms[:max_sequence_length]
        else:
            pad_len = max_sequence_length - len(aug_zcr)
            aug_zcr = np.pad(aug_zcr, (0, pad_len), "constant")
            aug_rms = np.pad(aug_rms, (0, pad_len), "constant")

        zcr_sequences.append(aug_zcr)
        rms_sequences.append(aug_rms)
        aug_labels.append(label)

# Konwersja na numpy arrays
X_zcr = np.array(zcr_sequences).reshape(-1, max_sequence_length, 1)
X_rms = np.array(rms_sequences).reshape(-1, max_sequence_length, 1)
y = np.array(aug_labels).astype("int64")

# Wyświetlanie informacji o kształcie danych
print(f"Kształt danych X_zcr: {X_zcr.shape}")
print(f"Kształt danych X_rms: {X_rms.shape}")
print(f"Kształt danych y: {y.shape}")
print(f"Liczba oryginalnych próbek: {len(audio_files)}")
print(f"Liczba próbek po augmentacji: {len(zcr_sequences)}")

# Podział na zbiory treningowy, walidacyjny i testowy
indices = np.arange(len(y))
indices_train, indices_temp, y_train, y_temp = train_test_split(
    indices, y, test_size=0.3, random_state=42, stratify=y
)
indices_val, indices_test, y_val, y_test = train_test_split(
    indices_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

X_zcr_train, X_zcr_val, X_zcr_test = (
    X_zcr[indices_train],
    X_zcr[indices_val],
    X_zcr[indices_test],
)
X_rms_train, X_rms_val, X_rms_test = (
    X_rms[indices_train],
    X_rms[indices_val],
    X_rms[indices_test],
)

print(f"Rozmiar zbioru treningowego: {len(y_train)} próbek")
print(f"Rozmiar zbioru walidacyjnego: {len(y_val)} próbek")
print(f"Rozmiar zbioru testowego: {len(y_test)} próbek")

# Wyświetl rozkład klas w zbiorze treningowym
class_counts = np.bincount(y_train)
print("\nRozkład klas w zbiorze treningowym:")
for label, count in enumerate(class_counts):
    emotion = [k for k, v in emotion_labels.items() if v == label][0]
    print(f" {emotion}: {count} próbek")

# Obliczenie wag klas dla zrównoważenia
class_weights = {}
total_samples = len(y_train)
n_classes = len(emotion_labels)
for label, count in enumerate(class_counts):
    class_weights[label] = total_samples / (n_classes * count)

print("\nWagi klas:")
for label, weight in class_weights.items():
    emotion = [k for k, v in emotion_labels.items() if v == label][0]
    print(f" {emotion}: {weight:.4f}")

# Przygotowanie danych w formacie PyTorch
X_zcr_train_tensor = torch.tensor(X_zcr_train, dtype=torch.float32)
X_rms_train_tensor = torch.tensor(X_rms_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_zcr_val_tensor = torch.tensor(X_zcr_val, dtype=torch.float32)
X_rms_val_tensor = torch.tensor(X_rms_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_zcr_test_tensor = torch.tensor(X_zcr_test, dtype=torch.float32)
X_rms_test_tensor = torch.tensor(X_rms_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Tworzenie TensorDatasets
train_dataset = TensorDataset(X_zcr_train_tensor, X_rms_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_zcr_val_tensor, X_rms_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_zcr_test_tensor, X_rms_test_tensor, y_test_tensor)

# DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Definiowanie modelu w PyTorch
class ParallelConv1DRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ParallelConv1DRNN, self).__init__()

        # Ścieżka Conv1D dla ZCR
        self.conv1_zcr = nn.Conv1d(1, 64, kernel_size=5, padding=2)
        self.bn1_zcr = nn.BatchNorm1d(64)
        self.pool1_zcr = nn.MaxPool1d(2)
        self.dropout1_zcr = nn.Dropout(0.3)

        self.conv2_zcr = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn2_zcr = nn.BatchNorm1d(128)
        self.pool2_zcr = nn.MaxPool1d(2)
        self.dropout2_zcr = nn.Dropout(0.4)

        self.global_avg_pool_zcr = nn.AdaptiveAvgPool1d(1)

        # Ścieżka RNN dla RMS
        self.lstm1_rms = nn.LSTM(
            input_size=1, hidden_size=hidden_size, batch_first=True, bidirectional=True
        )

        # POPRAWKA: Używamy hidden_size*2 zamiast max_sequence_length
        # Było: self.bn1_rms = nn.BatchNorm1d(max_sequence_length)
        self.bn1_rms = nn.BatchNorm1d(hidden_size * 2)  # POPRAWIONE

        self.dropout1_rms = nn.Dropout(0.3)

        self.lstm2_rms = nn.LSTM(
            input_size=hidden_size * 2,  # *2 for bidirectional
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True,
        )
        self.bn2_rms = nn.BatchNorm1d(hidden_size * 2)
        self.dropout2_rms = nn.Dropout(0.4)

        # Warstwy połączeniowe
        # 128 from Conv1D + 128 from Bidirectional LSTM (64*2)
        merged_size = 128 + hidden_size * 2
        self.fc1 = nn.Linear(merged_size, 128)
        self.bn_fc = nn.BatchNorm1d(128)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x_zcr, x_rms):
        # Ścieżka Conv1D dla ZCR
        # Transpozycja dla Conv1d (batch, channels, seq_len)
        x_zcr = x_zcr.permute(0, 2, 1)

        x_zcr = self.conv1_zcr(x_zcr)
        x_zcr = self.bn1_zcr(x_zcr)
        x_zcr = F.relu(x_zcr)
        x_zcr = self.pool1_zcr(x_zcr)
        x_zcr = self.dropout1_zcr(x_zcr)

        x_zcr = self.conv2_zcr(x_zcr)
        x_zcr = self.bn2_zcr(x_zcr)
        x_zcr = F.relu(x_zcr)
        x_zcr = self.pool2_zcr(x_zcr)
        x_zcr = self.dropout2_zcr(x_zcr)

        x_zcr = self.global_avg_pool_zcr(x_zcr)
        x_zcr = x_zcr.view(x_zcr.size(0), -1)  # Flatten

        # Ścieżka RNN dla RMS
        x_rms_out, _ = self.lstm1_rms(x_rms)

        # Permutacja dla BatchNorm1d (batch, features, seq_len)
        x_rms_bn = x_rms_out.permute(0, 2, 1)
        x_rms_bn = self.bn1_rms(x_rms_bn)
        x_rms_bn = x_rms_bn.permute(0, 2, 1)  # Back to (batch, seq_len, features)

        x_rms_bn = self.dropout1_rms(x_rms_bn)

        _, (h_n, _) = self.lstm2_rms(x_rms_bn)
        # h_n shape: (2, batch, hidden_size) - 2 for bidirectional

        # Concatenate the final hidden states from both directions
        x_rms = torch.cat((h_n[0], h_n[1]), dim=1)

        x_rms = self.bn2_rms(x_rms)
        x_rms = self.dropout2_rms(x_rms)

        # Połączenie cech
        x_combined = torch.cat((x_zcr, x_rms), dim=1)

        # Warstwy klasyfikacyjne
        x = self.fc1(x_combined)
        x = self.bn_fc(x)
        x = F.relu(x)
        x = self.dropout_fc(x)
        x = self.fc2(x)

        return x


# Inicjalizacja modelu
input_size = 1  # Pojedyncza cecha (ZCR lub RMS) w każdym kanale
hidden_size = 64  # Rozmiar warstw ukrytych
num_classes = len(emotion_labels)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Używane urządzenie: {device}")

# Tworzenie modelu
model = ParallelConv1DRNN(input_size, hidden_size, num_classes).to(device)

# Wyświetlenie liczby parametrów modelu
total_params = sum(p.numel() for p in model.parameters())
print(f"Liczba parametrów modelu: {total_params}")

# Kompilacja modelu
# Konwersja wag klas na tensor PyTorch
class_weights_tensor = torch.tensor(
    [class_weights[i] for i in range(len(class_weights))], dtype=torch.float32
).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Funkcja zmieniająca współczynnik uczenia w zależności od epoki
def lr_scheduler_func(epoch, initial_lr):
    if epoch < 5:
        return initial_lr
    elif epoch < 10:
        return initial_lr * 0.5
    elif epoch < 15:
        return initial_lr * 0.25
    else:
        return initial_lr * 0.1


# Early stopping
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                if self.restore_best_weights:
                    model.load_state_dict(self.best_weights)
                return True
        return False

    def save_checkpoint(self, model):
        self.best_weights = model.state_dict().copy()


# Historia treningu
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Trening
epochs = 25
initial_lr = 0.001
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

print("\n===== TRENING MODELU RÓWNOLEGŁEGO CONV1D I RNN =====")

for epoch in range(epochs):
    # Aktualizacja learning rate
    new_lr = lr_scheduler_func(epoch, initial_lr)
    for param_group in optimizer.param_groups:
        param_group["lr"] = new_lr

    # Trening
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch_zcr, batch_rms, batch_labels in train_loader:
        batch_zcr, batch_rms, batch_labels = (
            batch_zcr.to(device),
            batch_rms.to(device),
            batch_labels.to(device),
        )

        optimizer.zero_grad()
        outputs = model(batch_zcr, batch_rms)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += batch_labels.size(0)
        train_correct += predicted.eq(batch_labels).sum().item()

    # Walidacja
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch_zcr, batch_rms, batch_labels in val_loader:
            batch_zcr, batch_rms, batch_labels = (
                batch_zcr.to(device),
                batch_rms.to(device),
                batch_labels.to(device),
            )

            outputs = model(batch_zcr, batch_rms)
            loss = criterion(outputs, batch_labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += batch_labels.size(0)
            val_correct += predicted.eq(batch_labels).sum().item()

    # Obliczenie średnich
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total

    # Zapisanie historii
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(
        f"Epoka {epoch + 1}/{epochs}: "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, LR: {new_lr:.6f}"
    )

    # Early stopping
    if early_stopping(val_loss, model):
        print(f"Early stopping w epoce {epoch + 1}")
        break

# Zapisanie najlepszego modelu
torch.save(
    model.state_dict(), os.path.join(model_output_dir, "best_conv1d_rnn_model.pt")
)
print(
    f"Zapisano najlepszy model do pliku {os.path.join(model_output_dir, 'best_conv1d_rnn_model.pt')}"
)

# Ocena modelu na zbiorze testowym
model.eval()
test_loss = 0.0
test_correct = 0
test_total = 0
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_zcr, batch_rms, batch_labels in test_loader:
        batch_zcr, batch_rms, batch_labels = (
            batch_zcr.to(device),
            batch_rms.to(device),
            batch_labels.to(device),
        )

        outputs = model(batch_zcr, batch_rms)
        loss = criterion(outputs, batch_labels)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        test_total += batch_labels.size(0)
        test_correct += predicted.eq(batch_labels).sum().item()

        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

test_acc = test_correct / test_total
test_loss /= len(test_loader)
print(f"\nDokładność na zbiorze testowym: {test_acc:.4f}")
print(f"Strata na zbiorze testowym: {test_loss:.4f}")

y_pred_classes = np.array(all_predictions)
y_test_final = np.array(all_labels)

# Raport klasyfikacji
print("\nRaport klasyfikacji:")
print(classification_report(y_test_final, y_pred_classes))

# Macierz pomyłek
cm = confusion_matrix(y_test_final, y_pred_classes)
print("\nMacierz pomyłek:")
print(cm)

# Obliczanie F1-score
f1_macro = f1_score(y_test_final, y_pred_classes, average="macro")
print(f"\nMacro F1-score: {f1_macro:.4f}")

# Wykresy
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=[k for k in emotion_labels.keys()],
    yticklabels=[k for k in emotion_labels.keys()],
)
plt.xlabel("Przewidziana emocja")
plt.ylabel("Prawdziwa emocja")
plt.title("Macierz pomyłek")
plt.tight_layout()
plt.savefig(os.path.join(model_output_dir, "confusion_matrix.png"))
plt.close()

# Historia treningu
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label="Dokładność treningu")
plt.plot(val_accuracies, label="Dokładność walidacji")
plt.title("Dokładność modelu")
plt.xlabel("Epoka")
plt.ylabel("Dokładność")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses, label="Strata treningu")
plt.plot(val_losses, label="Strata walidacji")
plt.title("Strata modelu")
plt.xlabel("Epoka")
plt.ylabel("Strata")
plt.legend()

plt.tight_layout()
plt.savefig(os.path.join(model_output_dir, "training_history.png"))
plt.close()

# Zapisanie mapowania etykiet
with open(os.path.join(model_output_dir, "emotion_labels.json"), "w") as f:
    json.dump(emotion_labels, f)
print(
    f"Zapisano mapowanie etykiet do pliku {os.path.join(model_output_dir, 'emotion_labels.json')}"
)

# Znormalizowana macierz pomyłek
cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm_normalized,
    annot=True,
    fmt=".2f",
    cmap="Blues",
    xticklabels=list(emotion_labels.keys()),
    yticklabels=list(emotion_labels.keys()),
)
plt.xlabel("Etykiety przewidywane")
plt.ylabel("Etykiety rzeczywiste")
plt.title("Znormalizowana Macierz Pomyłek")
plt.savefig(
    os.path.join(model_output_dir, "normalized_confusion_matrix.png"),
    format="png",
    dpi=300,
)
plt.close()